In [10]:
import tensorflow as tf
import numpy as np
import os

In [11]:
print(f"TensorFlow 版本: {tf.__version__}")

INPUT_SHAPE = (96, 96, 3) # <--- 修改点
NUM_CLASSES = 10           # 假的分类头，10类 (数字不重要
MODEL_ALPHA = 0.25          # 宽度系数 

# --- 1. 定义模型和保存路径 ---
MODEL_DIR = f"models_mobilenet_v1_{INPUT_SHAPE[0]}"
FP32_SAVED_MODEL_PATH = os.path.join(MODEL_DIR, f"mobilenet_v1_{INPUT_SHAPE[0]}_fp32_savedmodel")
TFLITE_INT8_MODEL_PATH = os.path.join(MODEL_DIR, f"mobilenet_v1_{INPUT_SHAPE[0]}_int8.tflite")
# 确保目录存在
os.makedirs(MODEL_DIR, exist_ok=True)

TensorFlow 版本: 2.20.0


In [12]:
# --- 计算 SavedModel 总大小的函数 ---
def get_savedmodel_total_size(path):
    """计算 SavedModel 目录的总大小"""
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for filename in filenames:
            filepath = os.path.join(dirpath, filename)
            if os.path.isfile(filepath):
                total_size += os.path.getsize(filepath)
    return total_size

In [14]:
# --- 2. 加载预训练的 FP32 MobileNetV1 ---
# 我们需要一个基础模型来进行转换

print(f"正在加载 FP32 MobileNetV1 - {INPUT_SHAPE[0]}(ImageNet 预训练)...")
base_model = tf.keras.applications.MobileNet(
    alpha=MODEL_ALPHA,
    weights='imagenet', 
    include_top=False,
    input_shape=INPUT_SHAPE,
)

# 针对96x96输入，我们需要添加一个分类头
# 添加一个假的分类头 (因为我们只关心算子，不关心精度)
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling")(x)
x = tf.keras.layers.Dense(NUM_CLASSES, name="classifier")(x)

fp32_model = tf.keras.Model(inputs=base_model.input, outputs=x)

# print("96 x 96 model architecture:")
# fp32_model.summary()

# 保存为 SavedModel 格式，这是 TFLiteConverter 的标准输入
print(f"正在将 FP32 模型保存到: {FP32_SAVED_MODEL_PATH}")
fp32_model.export(FP32_SAVED_MODEL_PATH)
print("FP32 模型保存完毕。")

# --- 3. 创建一个“虚拟”的校准数据集 ---
#
# !!! 关键说明 !!!
# 正常情况下，这里需要 100-500 张你真实场景的图片。
# 因为你只关心算子优化，不关心精度，我们这里只使用
# 几批随机数据来“喂饱” TFLiteConverter，让它完成 INT8 的转换。
#
# MobileNetV1 的 Keras 预处理是将 [0, 255] 缩放到 [-1, 1]
# 所以我们生成这个范围内的随机数据。
#

def dummy_representative_dataset_gen():
    # 我们只提供 10 批数据，每批 1 张图片
    for _ in range(10):
        # 1. 生成 [0, 1] 范围的随机数据
        data = np.random.rand(1, *INPUT_SHAPE).astype(np.float32)
        # 2. 缩放到 [-1, 1] 范围
        data = (data * 2.0) - 1.0
        # 3. yield 一个列表
        yield [data]
    print("已生成所有“虚拟”校准数据。")
    
def representative_dataset_gen():
    # 使用正确的 MobileNet 预处理：生成 [0, 255] 范围的随机数据
    for _ in range(10):
        # 生成 [0, 255] 范围的随机图像数据
        data = np.random.rand(1, *INPUT_SHAPE).astype(np.float32) * 255.0
        yield [data]


# --- 4. 执行 INT8 转换 ---
print("开始执行 TFLite INT8 转换...")

# 1. 初始化 TFLiteConverter
converter = tf.lite.TFLiteConverter.from_saved_model(FP32_SAVED_MODEL_PATH)

# 2. 开启优化 (这是INT8量化的开关)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# 3. 指定“虚拟”校准数据集
# converter.representative_dataset = dummy_representative_dataset_gen
converter.representative_dataset = representative_dataset_gen

# 4. 强制要求所有算子都是 INT8
# 这是为了确保模型在你的板子上能最大程度地使用 INT8 硬件
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # 设置输入为 INT8
converter.inference_output_type = tf.int8 # 设置输出为 INT8

# 5. 执行转换
print("正在转换... 这可能需要几分钟时间...")
try:
    tflite_int8_model_content = converter.convert()
    print("模型转换成功！")

    # 6. 保存 INT8 TFLite 模型
    with open(TFLITE_INT8_MODEL_PATH, "wb") as f:
        f.write(tflite_int8_model_content)
    
    print("-" * 50)
    print(f"恭喜！纯 INT8 TFLite 模型已生成！")
    print(f"路径: {TFLITE_INT8_MODEL_PATH}")
    
    # 比较文件大小
    fp32_size_approx = get_savedmodel_total_size(FP32_SAVED_MODEL_PATH) / (1024*1024)
    int8_size = os.path.getsize(TFLITE_INT8_MODEL_PATH) / (1024*1024)
    
    print(f"FP32 SavedModel (pb文件) 大小约: {fp32_size_approx:.2f} MB")
    print(f"INT8 TFLite 模型大小: {int8_size:.2f} MB")
    print(f"(模型大小减少了约 {(1 - int8_size / fp32_size_approx) * 100:.1f}%)")
    print("-" * 50)

except Exception as e:
    print(f"模型转换失败: {e}")
    print("请检查你的 TensorFlow 和 NumPy 版本。")

正在加载 FP32 MobileNetV1 - 96(ImageNet 预训练)...


/tmp/ipykernel_6886/957121652.py:5: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNet(


正在将 FP32 模型保存到: models_mobilenet_v1_96/mobilenet_v1_96_fp32_savedmodel
INFO:tensorflow:Assets written to: models_mobilenet_v1_96/mobilenet_v1_96_fp32_savedmodel/assets


INFO:tensorflow:Assets written to: models_mobilenet_v1_96/mobilenet_v1_96_fp32_savedmodel/assets


Saved artifact at 'models_mobilenet_v1_96/mobilenet_v1_96_fp32_savedmodel'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name='keras_tensor_352')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  125775306958096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  125775306958288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  125775306950608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  125775306955792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  125775306956752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  125775306956944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  125775306958480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  125775306957904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  125775306954832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  125775306948688: TensorSpec(shape=(), dtype

W0000 00:00:1761244224.333586    6886 tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
W0000 00:00:1761244224.333648    6886 tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-10-23 20:30:24.333820: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: models_mobilenet_v1_96/mobilenet_v1_96_fp32_savedmodel
2025-10-23 20:30:24.337244: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-10-23 20:30:24.337271: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: models_mobilenet_v1_96/mobilenet_v1_96_fp32_savedmodel
2025-10-23 20:30:24.375016: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-10-23 20:30:24.624976: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: models_mobilenet_v1_96/mobilenet_v1_96_fp32_savedmodel
2025-10-23 20:30:24.690488: I tensorflow/cc/saved_model/loader.cc:471] SavedModel lo